# Baseline model without RAG

This is a baseline model for the QA task without the RAG pipline.

In order to compare, we test four models as the backbone model in the RAG pipeline: `meta-llama/Llama-3.2-3B-Instruct`, `meta-llama/Llama-3.1-8B-Instruct`, `gemini-2.0-flash`, and `gemini-2.0-flash-thinking-exp-01-21`. We also adopt the same data type (fp16) and the same config for setting up the tokenizer. We use the same prompt format as the one in the RAG pipeline.

### meta-llama/Llama-3.2-3B-Instruct model

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import login


model_name = "meta-llama/Llama-3.2-3B-Instruct"

login(token=os.getenv("HUGGINGFACE_TOKEN", "your_huggingface_token"))


model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

generation_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# Step 3: load qa annotation test set
import pandas as pd
# qa_df = pd.read_csv("../data/annotated/QA_pairs_1.csv")
qa_df = pd.read_csv("./data/test/test_questions.csv")

# doc_ids = qa_df["Doc_id"].tolist()
questions = qa_df["Question"].tolist()
# answers = qa_df["Reference_Answers"].tolist()

# # random sample 10 qa pairs
# import random
# sample_size = 10
# random.seed(747)
# sample_indices = random.sample(range(len(questions)), sample_size)
# sample_doc_ids = [doc_ids[i] for i in sample_indices]
# sample_questions = [questions[i] for i in sample_indices]
# sample_answers = [answers[i] for i in sample_indices]

In [ ]:
template = """
You are an expert assistant answering factual questions about various aspects of Pittsburgh or Carnegie Mellon University (CMU), including history, policy, culture, events, and more.
If you do not know the answer, just say "I don't know."

Important Instructions:
- Answer concisely without repeating the question.
- Do **not** use complete sentences. Provide only the word, name, date, or phrase that directly answers the question. For example, given the question "When was Carnegie Mellon University founded?", you should only answer "1900".

Examples:
Question: Who is Pittsburgh named after?
Answer: William Pitt
Question: What famous machine learning venue had its first conference in Pittsburgh in 1980?
Answer: ICML
Question: What musical artist is performing at PPG Arena on October 13?
Answer: Billie Eilish

Question: {question} \n\n
Answer:
"""

In [ ]:
# use the template to generate the answers
from tqdm import tqdm
generated_answers = []
for question in tqdm(questions):
    full_prompt = template.format(question=question)
    messages = [
        {"role": "user", "content": full_prompt},
        ]
    output = generation_pipe(messages, max_new_tokens=50)
    generated_answers.append(output[0]["generated_text"][1]['content'])

100%|██████████| 574/574 [02:04<00:00,  4.60it/s]


In [ ]:
# write all columns to a csv file
# results_df = pd.DataFrame({
#         "Doc_id": doc_ids,
#         "Question": questions,
#         "Reference_Answers": answers,
#         "Generated_Answer": generated_answers,
#     })

results_df = pd.DataFrame({
        "Question": questions,
        "Generated_Answer": generated_answers,
    })

# save the results to a csv file
results_df.to_csv("./output/closebook_baseline.csv", index=False)

In [ ]:
results_df

,Question,Generated_Answer
0,"What bank, which is the 5th largest in the US,...",PNC Bank
1,How many bridges does Pittsburgh have?,403
2,Who named the city of Pittsburgh?,General Robert Moore
3,At what park do the three rivers converge in P...,Point State Park
4,How many neighborhoods does Pittsburgh have?,19
...,...,...
569,What is the primary focus of the event at the ...,Pittsburgh JazzLive
570,Where and when is the Pittsburgh Veg Fair held...,Pennsylvania State Farm Show Complex
571,How can restaurants get involved with Pittsbur...,Register online through VisitPittsburgh
572,What are the benefits of sponsoring the Pittsb...,I don't know



### meta-llama/Llama-3.1-8B-Instruct

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import login


model_name = "meta-llama/Llama-3.1-8B-Instruct"

login(token=os.getenv("HUGGINGFACE_TOKEN", "your_huggingface_token"))

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

generation_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16
)

In [ ]:
# Step 3: load qa annotation test set
import pandas as pd
qa_df = pd.read_csv("./data/annotated/QA_pairs_1.csv")
#qa_df = pd.read_csv("./data/test/test_questions.csv")

doc_ids = qa_df["Doc_id"].tolist()
questions = qa_df["Question"].tolist()
answers = qa_df["Reference_Answers"].tolist()

# random sample 10 qa pairs
import random
sample_size = 100
random.seed(747)
sample_indices = random.sample(range(len(questions)), sample_size)
sample_doc_ids = [doc_ids[i] for i in sample_indices]
sample_questions = [questions[i] for i in sample_indices]
sample_answers = [answers[i] for i in sample_indices]

In [ ]:
template = """
You are an expert assistant answering factual questions about various aspects of Pittsburgh or Carnegie Mellon University (CMU), including history, policy, culture, events, and more.
If you do not know the answer, just say "I don't know."

Important Instructions:
- Answer concisely without repeating the question.
- Do **not** use complete sentences. Provide only the word, name, date, or phrase that directly answers the question. For example, given the question "When was Carnegie Mellon University founded?", you should only answer "1900".

Examples:
Question: Who is Pittsburgh named after?
Answer: William Pitt
Question: What famous machine learning venue had its first conference in Pittsburgh in 1980?
Answer: ICML
Question: What musical artist is performing at PPG Arena on October 13?
Answer: Billie Eilish

Question: {question} \n\n
Answer:
"""

In [ ]:
# use the template to generate the answers
from tqdm import tqdm
generated_answers = []
for question in tqdm(questions):
    full_prompt = template.format(question=question)
    messages = [
        {"role": "user", "content": full_prompt},
        ]
    output = generation_pipe(messages, max_new_tokens=50)
    generated_answers.append(output[0]["generated_text"][1]['content'])

In [ ]:
# write all columns to a csv file
results_df = pd.DataFrame({
        "Doc_id": doc_ids,
        "Question": questions,
        "Reference_Answers": answers,
        "Generated_Answer": generated_answers,
    })

# results_df = pd.DataFrame({
#         "Question": questions,
#         "Generated_Answer": generated_answers,
#     })

# save the results to a csv file
results_df.to_csv("./output/llama31_8B_baseline.csv", index=False)

In [ ]:
results_df

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import login
import pandas as pd
import random
from tqdm import tqdm

def run_llama_baseline(model_name, output_file, qa_file="./data/annotated/QA_pairs_1.csv", sample_size=100):
    """
    Generate QA baseline answers using a specified model and save the results to a CSV file.

    Args:
        model_name (str): Name of the Hugging Face model to use.
        output_file (str): Path to save the generated answers.
        qa_file (str): Path to the QA annotation test set CSV file.
        sample_size (int): Number of QA pairs to sample for evaluation.
    """
    # Login to Hugging Face Hub
    login(token=os.getenv("HUGGINGFACE_TOKEN", "your_huggingface_token"))

    # Load the model and tokenizer
    print(f"Loading model: {model_name}")
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"

    # Initialize the text generation pipeline
    generation_pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16
    )

    # Load QA annotation test set
    print(f"Loading QA file: {qa_file}")
    # Randomly sample QA pairs
    random.seed(42)
    qa_df = pd.read_csv(qa_file)
    qa_df = qa_df.sample(sample_size, random_state=221)
    sample_doc_ids = qa_df["Doc_id"].tolist()
    sample_questions = qa_df["Question"].tolist()
    sample_answers = qa_df["Reference_Answers"].tolist()

    #sample_indices = random.sample(range(len(questions)), sample_size)

    #sample_doc_ids = [doc_ids[i] for i in sample_indices]
    #sample_questions = [questions[i] for i in sample_indices]
    #sample_answers = [answers[i] for i in sample_indices]

    # Define the prompt template
    template = """
    You are an expert assistant answering factual questions about various aspects of Pittsburgh or Carnegie Mellon University (CMU), including history, policy, culture, events, and more.
    If you do not know the answer, just say "I don't know."

    Important Instructions:
    - Answer concisely without repeating the question.
    - Do **not** use complete sentences. Provide only the word, name, date, or phrase that directly answers the question. For example, given the question "When was Carnegie Mellon University founded?", you should only answer "1900".

    Examples:
    Question: Who is Pittsburgh named after?
    Answer: William Pitt
    Question: What famous machine learning venue had its first conference in Pittsburgh in 1980?
    Answer: ICML
    Question: What musical artist is performing at PPG Arena on October 13?
    Answer: Billie Eilish

    Question: {question} \n\n
    Answer:
    """

    # Generate answers
    print("Generating answers...")
    generated_answers = []
    for question in tqdm(sample_questions):
        full_prompt = template.format(question=question)
        messages = [
            {"role": "user", "content": full_prompt},
        ]
        output = generation_pipe(messages, max_new_tokens=50)
        generated_answers.append(output[0]["generated_text"][1]['content'])

    # Save results to a CSV file
    results_df = pd.DataFrame({
        "Doc_id": sample_doc_ids,
        "Question": sample_questions,
        "Reference_Answers": sample_answers,
        "Generated_Answer": generated_answers,
    })
    results_df.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")

In [4]:
# Define the models and output file names
models = [
    ("meta-llama/Llama-3.2-3B-Instruct", "./output/llama32_3B_baseline.csv"),
    ("meta-llama/Llama-3.1-8B-Instruct", "./output/llama31_8B_baseline.csv")#,
    #("meta-llama/Llama-3.3-70B-Instruct", "./output/llama33_70B_baseline.csv")
]

# Run the function for each model
for model_name, output_file in models:
    run_llama_baseline(model_name, output_file)

Loading model: meta-llama/Llama-3.1-8B-Instruct


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

...

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


Loading QA file: ./data/annotated/QA_pairs_1.csv
Generating answers...


100%|██████████| 100/100 [00:24<00:00,  4.09it/s]


Results saved to ./output/llama31_8B_baseline.csv


### meta-llama/Llama-3.3-70B-Instruct

In [ ]:
run_baseline("meta-llama/Llama-3.3-70B-Instruct", "./output/llama33_70B_baseline.csv")

Loading model: meta-llama/Llama-3.3-70B-Instruct


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/59.6k [00:00<?, ?B/s]

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

...

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

Device set to use cuda:0


Loading QA file: ./data/annotated/QA_pairs_1.csv
Generating answers...


  0%|          | 0/100 [00:00<?, ?it/s]

Goocle Colab Pro A100 can't run llama3.3 70B, maybe Pro+ can. IDK

### gemini-2.0-flash

In [ ]:
import os
import pandas as pd
import random
from tqdm import tqdm
from google import genai
import time
from google import genai

import time
from google import genai

def call_gemini_with_interval(client, model, contents, config, interval=20, retries=3, backoff_factor=2):
    """
    Call Google Gemini API with a fixed interval between requests and retry logic for handling errors.

    Args:
        client (genai.Client): The Google Gemini client.
        model (str): The model name (e.g., "gemini-2.0-flash").
        contents (str): The input prompt or question.
        config (dict): Configuration for the API call (e.g., tools, temperature, etc.).
        interval (int): Time in seconds to wait between API requests.
        retries (int): Number of retries for handling quota exhaustion or other errors.
        backoff_factor (int): Factor for exponential backoff in case of retries.

    Returns:
        response: The response from the Google Gemini API.
    """
    for attempt in range(retries):
        try:
            # Send the API request
            response = client.models.generate_content(
                model=model,
                contents=contents,
                config=config,
            )
            print(f"Request succeeded. Waiting {interval} seconds before the next request...")
            time.sleep(interval)  # Wait for the specified interval
            return response
        except Exception as e:
            if "RESOURCE_EXHAUSTED" in str(e):
                print(f"Quota exceeded. Retrying in {backoff_factor ** attempt} seconds...")
                time.sleep(backoff_factor ** attempt)
            else:
                raise e
    raise Exception("Max retries exceeded. Quota still exhausted.")



def run_gemini_baseline(api_key, output_file, qa_file="./data/annotated/QA_pairs_1.csv", sample_size=100):
    """
    Generate QA baseline answers using Google's Gemini model and save the results to a CSV file.

    Args:
        api_key (str): API key for accessing Google's Gemini model.
        output_file (str): Path to save the generated answers.
        qa_file (str): Path to the QA annotation test set CSV file.
        sample_size (int): Number of QA pairs to sample for evaluation.
    """
    # Initialize the Gemini client
    client = genai.Client(api_key=api_key)

    # Load QA annotation test set
    print(f"Loading QA file: {qa_file}")
    # Randomly sample QA pairs
    random.seed(42)
    qa_df = pd.read_csv(qa_file)
    qa_df = qa_df.sample(sample_size, random_state=221)
    sample_doc_ids = qa_df["Doc_id"].tolist()
    sample_questions = qa_df["Question"].tolist()
    sample_answers = qa_df["Reference_Answers"].tolist()

    # Define the prompt template
    template = """
    You are an expert assistant answering factual questions about various aspects of Pittsburgh or Carnegie Mellon University (CMU), including history, policy, culture, events, and more.
    If you do not know the answer, just say "I don't know."

    Important Instructions:
    - Answer concisely without repeating the question.
    - Do **not** use complete sentences. Provide only the word, name, date, or phrase that directly answers the question. For example, given the question "When was Carnegie Mellon University founded?", you should only answer "1900".

    Examples:
    Question: Who is Pittsburgh named after?
    Answer: William Pitt
    Question: What famous machine learning venue had its first conference in Pittsburgh in 1980?
    Answer: ICML
    Question: What musical artist is performing at PPG Arena on October 13?
    Answer: Billie Eilish

    Question: {question} \n\n
    Answer:
    """

    # Generate answers
    print("Generating answers using Google Gemini...")
    generated_answers = []
    for question in tqdm(sample_questions):
        full_prompt = template.format(question=question)
        response = call_gemini_with_interval(
          client,
          model="gemini-2.0-flash-thinking-exp-01-21",
          contents=full_prompt,
          config={}, # {"tools": [{"google_search": {}}]},
          interval=4,  # Wait 4 seconds between requests
          retries=3,    # Retry up to 3 times if quota is exceeded
          backoff_factor=2  # Exponential backoff for retries
        )

        # Extract the generated answer
        generated_answers.append(response.text)

    # Save results to a CSV file
    results_df = pd.DataFrame({
        "Doc_id": sample_doc_ids,
        "Question": sample_questions,
        "Reference_Answers": sample_answers,
        "Generated_Answer": generated_answers,
    })
    results_df.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")

In [ ]:
# Example usage
api_key = "your gemini API key"
output_file = "./output/gemini_2_flash_thinking_baseline.csv"
qa_file = "./data/annotated/QA_pairs_1.csv"
sample_size = 100

run_gemini_baseline(api_key, output_file, qa_file, sample_size)

Loading QA file: ./data/annotated/QA_pairs_1.csv
Generating answers using Google Gemini...


  0%|          | 0/100 [00:00<?, ?it/s]

Request succeeded. Waiting 4 seconds before the next request...


  1%|          | 1/100 [00:05<09:48,  5.94s/it]

Request succeeded. Waiting 4 seconds before the next request...


...

Request succeeded. Waiting 4 seconds before the next request...


 99%|█████████▉| 99/100 [10:17<00:05,  5.62s/it]

Request succeeded. Waiting 4 seconds before the next request...


100%|██████████| 100/100 [10:28<00:00,  6.28s/it]

Results saved to ./output/gemini_2_flash_thinking_baseline.csv


In [ ]:
import os
import pandas as pd
import random
from tqdm import tqdm
from google import genai
def call_gemini_with_interval(client, model, contents, config, interval=20, retries=3, backoff_factor=2):
    """
    Call Google Gemini API with a fixed interval between requests and retry logic for handling errors.

    Args:
        client (genai.Client): The Google Gemini client.
        model (str): The model name (e.g., "gemini-2.0-flash").
        contents (str): The input prompt or question.
        config (dict): Configuration for the API call (e.g., tools, temperature, etc.).
        interval (int): Time in seconds to wait between API requests.
        retries (int): Number of retries for handling quota exhaustion or other errors.
        backoff_factor (int): Factor for exponential backoff in case of retries.

    Returns:
        response: The response from the Google Gemini API.
    """
    for attempt in range(retries):
        try:
            # Send the API request
            response = client.models.generate_content(
                model=model,
                contents=contents,
                config=config,
            )
            print(f"Request succeeded. Waiting {interval} seconds before the next request...")
            time.sleep(interval)  # Wait for the specified interval
            return response
        except Exception as e:
            if "RESOURCE_EXHAUSTED" in str(e):
                print(f"Quota exceeded. Retrying in {backoff_factor ** attempt} seconds...")
                time.sleep(backoff_factor ** attempt)
            else:
                raise e
    raise Exception("Max retries exceeded. Quota still exhausted.")



def run_gemini_baseline_with_search(api_key, output_file, qa_file="./data/annotated/QA_pairs_1.csv", sample_size=100):
    """
    Generate QA baseline answers using Google's Gemini model with Google Search grounding and save the results to a CSV file.

    Args:
        api_key (str): API key for accessing Google's Gemini model.
        output_file (str): Path to save the generated answers.
        qa_file (str): Path to the QA annotation test set CSV file.
        sample_size (int): Number of QA pairs to sample for evaluation.
    """
    # Initialize the Gemini client
    client = genai.Client(api_key=api_key)

    # Load QA annotation test set
    print(f"Loading QA file: {qa_file}")
    # Randomly sample QA pairs
    random.seed(42)
    qa_df = pd.read_csv(qa_file)
    qa_df = qa_df.sample(sample_size, random_state=221)
    sample_doc_ids = qa_df["Doc_id"].tolist()
    sample_questions = qa_df["Question"].tolist()
    sample_answers = qa_df["Reference_Answers"].tolist()

    # Define the prompt template
    template = """
    You are an expert assistant answering factual questions about various aspects of Pittsburgh or Carnegie Mellon University (CMU), including history, policy, culture, events, and more.
    If you do not know the answer, just say "I don't know."

    Important Instructions:
    - Answer concisely without repeating the question.
    - Do **not** use complete sentences. Provide only the word, name, date, or phrase that directly answers the question. For example, given the question "When was Carnegie Mellon University founded?", you should only answer "1900".

    Examples:
    Question: Who is Pittsburgh named after?
    Answer: William Pitt
    Question: What famous machine learning venue had its first conference in Pittsburgh in 1980?
    Answer: ICML
    Question: What musical artist is performing at PPG Arena on October 13?
    Answer: Billie Eilish

    Question: {question} \n\n
    Answer:
    """

    # Generate answers
    print("Generating answers using Google Gemini with Google Search grounding...")
    generated_answers = []
    search_queries = []
    search_pages = []

    for question in tqdm(sample_questions):
        full_prompt = template.format(question=question)
        response = call_gemini_with_interval(
          client,
          model="gemini-2.0-flash-thinking-exp-01-21",
          contents=full_prompt,
          config={"tools": [{"google_search": {}}]}, # Enable Google Search grounding
          interval=4,  # Wait 4 seconds between requests
          retries=3,    # Retry up to 3 times if quota is exceeded
          backoff_factor=2  # Exponential backoff for retries
        )

        # Extract the generated answer
        generated_answers.append(response.text)

    # Save results to a CSV file
    results_df = pd.DataFrame({
        "Doc_id": sample_doc_ids,
        "Question": sample_questions,
        "Reference_Answers": sample_answers,
        "Generated_Answer": generated_answers
    })
    results_df.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")

In [ ]:
# Example usage
api_key = "your gemini API key"
output_file = "./output/gemini_baseline_with_search.csv"
qa_file = "./data/annotated/QA_pairs_1.csv"
sample_size = 100

run_gemini_baseline_with_search(api_key, output_file, qa_file, sample_size)

Loading QA file: ./data/annotated/QA_pairs_1.csv
Generating answers using Google Gemini with Google Search grounding...


  0%|          | 0/100 [00:00<?, ?it/s]

Request succeeded. Waiting 4 seconds before the next request...


  1%|          | 1/100 [00:06<10:48,  6.55s/it]

Request succeeded. Waiting 4 seconds before the next request...


...

Request succeeded. Waiting 4 seconds before the next request...


 99%|█████████▉| 99/100 [08:16<00:04,  4.68s/it]

Request succeeded. Waiting 4 seconds before the next request...


100%|██████████| 100/100 [08:21<00:00,  5.02s/it]

Results saved to ./output/gemini_baseline_with_search.csv


Add Search to Google Gemini didn't improve the accuracy

### gemini-2.0-flash-thinking-exp-01-21 and gemini-2.0-flash

run llama or gemini

In [ ]:
import os
import pandas as pd
import random
from tqdm import tqdm
from google import genai
import time
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import login


def call_gemini_with_interval(client, model, contents, config, interval=20, retries=3, backoff_factor=2):
    """
    Call Google Gemini API with a fixed interval between requests and retry logic for handling errors.

    Args:
        client (genai.Client): The Google Gemini client.
        model (str): The model name (e.g., "gemini-2.0-flash").
        contents (str): The input prompt or question.
        config (dict): Configuration for the API call (e.g., tools, temperature, etc.).
        interval (int): Time in seconds to wait between API requests.
        retries (int): Number of retries for handling quota exhaustion or other errors.
        backoff_factor (int): Factor for exponential backoff in case of retries.

    Returns:
        response: The response from the Google Gemini API.
    """
    for attempt in range(retries):
        try:
            # Send the API request
            response = client.models.generate_content(
                model=model,
                contents=contents,
                config=config,
            )
            print(f"Request succeeded. Waiting {interval} seconds before the next request...")
            time.sleep(interval)  # Wait for the specified interval
            return response
        except Exception as e:
            if "RESOURCE_EXHAUSTED" in str(e):
                print(f"Quota exceeded. Retrying in {backoff_factor ** attempt} seconds...")
                time.sleep(backoff_factor ** attempt)
            else:
                raise e
    raise Exception("Max retries exceeded. Quota still exhausted.")


def run_baseline(
    model_name,
    output_file,
    qa_file="./data/annotated/QA_pairs_1.csv",
    sample_size=100
):
    """
    Generate QA baseline answers using either Hugging Face LLaMA or Google Gemini and save the results to a CSV file.

    Args:
        model_name (str): Model name for LLaMA or Gemini (e.g., "gemini-2.0-flash" or "meta-llama/Llama-3.1-8B-Instruct").
        output_file (str): Path to save the generated answers.
        qa_file (str): Path to the QA annotation test set CSV file.
        sample_size (int): Number of QA pairs to sample for evaluation.
    """
    # Load QA annotation test set
    print(f"Loading QA file: {qa_file}")
    qa_df = pd.read_csv(qa_file)
    qa_df = qa_df.sample(sample_size, random_state=221)
    sample_doc_ids = qa_df["Doc_id"].tolist()
    sample_questions = qa_df["Question"].tolist()
    sample_answers = qa_df["Reference_Answers"].tolist()

    # Define the prompt template
    template = """
    You are an expert assistant answering factual questions about various aspects of Pittsburgh or Carnegie Mellon University (CMU), including history, policy, culture, events, and more.
    If you do not know the answer, just say "I don't know."

    Important Instructions:
    - Answer concisely without repeating the question.
    - Do **not** use complete sentences. Provide only the word, name, date, or phrase that directly answers the question. For example, given the question "When was Carnegie Mellon University founded?", you should only answer "1900".

    Examples:
    Question: Who is Pittsburgh named after?
    Answer: William Pitt
    Question: What famous machine learning venue had its first conference in Pittsburgh in 1980?
    Answer: ICML
    Question: What musical artist is performing at PPG Arena on October 13?
    Answer: Billie Eilish

    Question: {question} \n\n
    Answer:
    """

    # Determine the model type based on the model name
    if "gemini" in model_name.lower():
        # Initialize the Gemini client
        print("Initializing Google Gemini client...")
        client = genai.Client(api_key=os.getenv("GEMINI_API_KEY", "your_api_key_here")) #  

        # Generate answers using Google Gemini
        print("Generating answers using Google Gemini...")
        generated_answers = []
        for question in tqdm(sample_questions):
            full_prompt = template.format(question=question)
            response = call_gemini_with_interval(
                client,
                model=model_name,  # Use the provided Gemini model name
                contents=full_prompt,
                config={},  # {"tools": [{"google_search": {}}]},
                interval=4,  # Wait 4 seconds between requests
                retries=3,  # Retry up to 3 times if quota is exceeded
                backoff_factor=2  # Exponential backoff for retries
            )
            # Extract the generated answer
            generated_answers.append(response.text)

    elif "llama" in model_name.lower():
        # Login to Hugging Face Hub
        print("Initializing Hugging Face LLaMA model...")
        login(token=os.getenv("HUGGINGFACE_TOKEN", "your_huggingface_token_here")) # 

        # Load the model and tokenizer
        print(f"Loading LLaMA model: {model_name}")
        model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "left"

        # Initialize the text generation pipeline
        generation_pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            torch_dtype=torch.float16
        )

        # Generate answers using LLaMA
        print("Generating answers using LLaMA...")
        generated_answers = []
        for question in tqdm(sample_questions):
            full_prompt = template.format(question=question)
            messages = [
                {"role": "user", "content": full_prompt},
            ]
            output = generation_pipe(messages, max_new_tokens=50)
            generated_answers.append(output[0]["generated_text"][1]['content'])

    else:
        raise ValueError("Invalid model_name. Please ensure it contains 'gemini' or 'llama'.")

    # Save results to a CSV file
    results_df = pd.DataFrame({
        "Doc_id": sample_doc_ids,
        "Question": sample_questions,
        "Reference_Answers": sample_answers,
        "Generated_Answer": generated_answers,
    })
    results_df.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")

In [ ]:
run_baseline(
    model_name="meta-llama/Llama-3.2-3B-Instruct",
    output_file="./output/llama32_3B_baseline.csv",
    qa_file="./data/annotated/QA_pairs_1.csv",
    sample_size=100
)

In [4]:
run_baseline(
    model_name="gemini-2.0-flash-thinking-exp-01-21",
    output_file="./output/gemini_2_flash_thinking_baseline.csv",
    #qa_file="./data/annotated/QA_pairs_1.csv",
    #sample_size=100
)

Loading QA file: ./data/annotated/QA_pairs_1.csv
Initializing Google Gemini client...
Generating answers using Google Gemini...


  0%|          | 0/100 [00:00<?, ?it/s]

Request succeeded. Waiting 4 seconds before the next request...


  1%|          | 1/100 [00:09<15:28,  9.38s/it]

Request succeeded. Waiting 4 seconds before the next request...


...

Request succeeded. Waiting 4 seconds before the next request...


 99%|█████████▉| 99/100 [10:46<00:07,  7.52s/it]

Request succeeded. Waiting 4 seconds before the next request...


100%|██████████| 100/100 [10:51<00:00,  6.52s/it]


Results saved to ./output/gemini_2_flash_thinking_baseline.csv


In [ ]:
# Define the models and output file names
models = [
    ("meta-llama/Llama-3.2-3B-Instruct", "./output/llama32_3B_baseline.csv"),
    ("meta-llama/Llama-3.1-8B-Instruct", "./output/llama31_8B_baseline.csv"),
    ("gemini-2.0-flash", "./output/gemini_2_flash_baseline.csv"),
    ("gemini-2.0-flash-thinking-exp-01-21", "./output/gemini_2_flash_thinking_baseline.csv")
]

# Run the function for each model
for model_name, output_file in models:
    print(f"Running baseline for model: {model_name}")
    run_baseline(
        model_name=model_name,
        output_file=output_file#,
        #qa_file="./data/annotated/QA_pairs_1.csv",
        #sample_size=100
    )
    print(f"Completed baseline for model: {model_name}\n")

Running baseline for model: meta-llama/Llama-3.2-3B-Instruct
Loading QA file: ./data/annotated/QA_pairs_1.csv
Initializing Hugging Face LLaMA model...
Loading LLaMA model: meta-llama/Llama-3.2-3B-Instruct


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


Generating answers using LLaMA...


100%|██████████| 100/100 [00:21<00:00,  4.55it/s]


Results saved to ./output/llama31_8B_baseline.csv
Completed baseline for model: meta-llama/Llama-3.1-8B-Instruct

